In [1]:
!pip install tensorflow==2.8.0 keras==2.8.0
!pip install segmentation-models


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 497.6/497.6 MB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 52.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 51.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.5/462.5 kB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 38.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 781.3/781.3 kB 25.8 MB/s eta 0:00:00
  Attempting uninstall: keras
    Found existing installation: keras 2.15.0
    Uninstalling keras-2.15.0:
      Successfully uninstalled keras-2.15.0
  Attempting uninstall: tensorboard-data-server
    Found existing installation: tensorboard-data-server 0.7.2
    Uninstalling tensorboard-data-server-0.7.2:
      Successfully uninstalled tensorboard-data-server-0.7.2
  Attempting uninstall: google-auth-oauthlib
    Found existing 

In [2]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import os
import pickle
import matplotlib.patches as patches
import re
import random
from sklearn.model_selection import train_test_split
import cv2
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
# keras.utils.generic_utils = keras.utils
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import plot_model
from PIL import Image
import tensorflow as tf
import keras
from keras import backend as K
from keras.models import Model,load_model
from keras.regularizers import l2
import datetime
%load_ext tensorboard
import segmentation_models
from segmentation_models import Unet
from segmentation_models import get_preprocessing
import imgaug.augmenters as iaa
import segmentation_models as sm
sm.set_framework('tf.keras')
sm.framework()
from tensorflow.keras.losses import binary_crossentropy

Segmentation Models: using `keras` framework.


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
!pip install kaggle
!mkdir -p ~/.kaggle
!rm -rf /content/steel/
!cp /content/drive/MyDrive/kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle competitions download -c severstal-steel-defect-detection
!7z x -bsp2 -bd severstal-steel-defect-detection.zip -o/content/steel
contents = os.listdir('/content/steel/')

 98% 1.54G/1.57G [00:12<00:00, 197MB/s]
100% 1.57G/1.57G [00:12<00:00, 133MB/s]

7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.30GHz (306F0),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan         1 file, 1684204253 bytes (1607 MiB)

Extracting archive: severstal-steel-defect-detection.zip
 22% 4096 Open              --
Path = severstal-steel-defect-detection.zip
Type = zip
Physical Size = 1684204253

  0%      0% 197 - test_images/08e148869.jpg                                      1% 391 - test_images/122b15b9a.jpg                                      2% 580 - test_images/1b5984acb.jpg                                    

In [5]:
with open('/content/drive/MyDrive/data.pkl','rb') as f:
  train=pickle.load(f)

In [6]:
train.head()

,image_id,rle_1,rle_2,rle_3,rle_4,defect,stratify,defect_1,defect_2,defect_3,defect_4,total_defects
0,0002cc93b.jpg,29102 12 29346 24 29602 24 29858 24 30114 24 3...,,,,1,1,1,0,0,0,1
1,00031f466.jpg,,,,,0,0,0,0,0,0,0
2,000418bfc.jpg,,,,,0,0,0,0,0,0,0
3,000789191.jpg,,,,,0,0,0,0,0,0,0
4,0007a71bf.jpg,,,18661 28 18863 82 19091 110 19347 110 19603 11...,,1,3,0,0,1,0,1


In [7]:
train.shape

(12568, 12)

In [99]:
#https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html
x_train,x_test=train_test_split(train,test_size=0.10,stratify=train['stratify'],random_state=0)
x_train,x_val=train_test_split(x_train,test_size=0.20,stratify=x_train['stratify'],random_state=0)
print("x_train {}".format(x_train.shape)," x_val {}".format(x_val.shape)," x_test {}".format(x_test.shape))
print("="*100)
train_1=x_train[x_train['defect_1']==1][['image_id','rle_1']].rename(columns={'image_id':'image_id','rle_1':'rle'})
train_2=x_train[x_train['defect_2']==1][['image_id','rle_2']].rename(columns={'image_id':'image_id','rle_2':'rle'})
train_3=x_train[x_train['defect_3']==1][['image_id','rle_3']].rename(columns={'image_id':'image_id','rle_3':'rle'})
train_4=x_train[x_train['defect_4']==1][['image_id','rle_4']].rename(columns={'image_id':'image_id','rle_4':'rle'})
print("train_1 {}".format(train_1.shape)," train_2 {}".format(train_2.shape)," train_3 {}".format(train_3.shape)," train_4 {}".format(train_4.shape))
print("="*100)
val_1=x_val[x_val['defect_1']==1][['image_id','rle_1']].rename(columns={'image_id':'image_id','rle_1':'rle'})
val_2=x_val[x_val['defect_2']==1][['image_id','rle_2']].rename(columns={'image_id':'image_id','rle_2':'rle'})
val_3=x_val[x_val['defect_3']==1][['image_id','rle_3']].rename(columns={'image_id':'image_id','rle_3':'rle'})
val_4=x_val[x_val['defect_4']==1][['image_id','rle_4']].rename(columns={'image_id':'image_id','rle_4':'rle'})
print("val_1 {}".format(val_1.shape)," val_2 {}".format(val_2.shape)," val_3 {}".format(val_3.shape)," val_4 {}".format(val_4.shape))
print("="*100)
test_1=x_test[x_test['defect_1']==1][['image_id','rle_1']].rename(columns={'image_id':'image_id','rle_1':'rle'})
test_2=x_test[x_test['defect_2']==1][['image_id','rle_2']].rename(columns={'image_id':'image_id','rle_2':'rle'})
test_3=x_test[x_test['defect_3']==1][['image_id','rle_3']].rename(columns={'image_id':'image_id','rle_3':'rle'})
test_4=x_test[x_test['defect_4']==1][['image_id','rle_4']].rename(columns={'image_id':'image_id','rle_4':'rle'})
print("test_1 {}".format(test_1.shape)," test_2 {}".format(test_2.shape)," test_3 {}".format(test_3.shape)," test_4 {}".format(test_4.shape))

x_train (9048, 12)  x_val (2263, 12)  x_test (1257, 12)
train_1 (646, 2)  train_2 (174, 2)  train_3 (3714, 2)  train_4 (581, 2)
val_1 (161, 2)  val_2 (50, 2)  val_3 (924, 2)  val_4 (142, 2)
test_1 (90, 2)  test_2 (23, 2)  test_3 (512, 2)  test_4 (78, 2)


In [48]:
#https://stackoverflow.com/questions/31273652/how-to-calculate-dice-coefficient-for-measuring-accuracy-of-image-segmentation-i
def dice_coef(y_true,y_pred):
  y_true_f=tf.reshape(tf.dtypes.cast(y_true, tf.float32),[-1])
  y_pred_f=tf.reshape(tf.dtypes.cast(y_pred, tf.float32),[-1])
  intersection=tf.reduce_sum(y_true_f*y_pred_f)
  return (2. * intersection + 1.) / (tf.reduce_sum(y_true_f) + tf.reduce_sum(y_pred_f) + 1.)

#https://stackoverflow.com/questions/49785133/keras-dice-coefficient-loss-function-is-negative-and-increasing-with-epochs
def dice_loss(y_true, y_pred):
  y_true_f = tf.reshape(y_true, [-1])
  y_pred_f = tf.reshape(y_pred, [-1])
  return (1-dice_coefficient(y_true, y_pred))

#defining function for calculation of loss function: binary cross entropy + dice loss
def bce_dice_loss(y_true, y_pred):
  y_true_f = tf.reshape(y_true, [-1])
  y_pred_f = tf.reshape(y_pred, [-1])
  return binary_crossentropy(y_true, y_pred) + (1-dice_coef(y_true, y_pred))

#https://www.kaggle.com/paulorzp/rle-functions-run-lenght-encode-decode
def rle_to_mask(rle):
  # CONVERT RLE TO MASK
  if (pd.isnull(rle))|(rle=='')|(rle=='-1'):
    return np.zeros((256,800),dtype=np.uint8) #If the EncodedPixels string is empty an empty mask is returned
  height=256
  width=1600
  mask=np.zeros(width*height,dtype=np.uint8)

  array=np.asarray([int(x) for x in rle.split()])
  starts=array[0::2]-1
  lengths=array[1::2]
  for index,start in enumerate(starts):
    mask[int(start):int(start+lengths[index])]=1

  return mask.reshape((height,width),order='F')[::,::2]

In [134]:
class train_DataGenerator(keras.utils.all_utils.Sequence):
  def __init__(self,dataframe,batch_size=4,shuffle=True,preprocess=None,info={}):
    self.batch_size = batch_size
    self.df = dataframe
    self.indices = self.df.index.tolist()
    self.preprocess = preprocess
    self.shuffle = shuffle
    self.on_epoch_end()

  def __len__(self):
    return len(self.indices) // (self.batch_size)

  def __getitem__(self, index):
    index = self.index[index * self.batch_size:(index + 1) * self.batch_size]
    batch = [self.indices[k] for k in index]

    X, y = self.__get_data(batch)
    return X, y

  def on_epoch_end(self):
    self.index = np.arange(len(self.indices))
    if self.shuffle == True:
      np.random.shuffle(self.index)

  def __get_data(self, batch):
    train_datagen = ImageDataGenerator()
    #https://www.geeksforgeeks.org/python-select-random-value-from-a-list/
    X=np.empty((self.batch_size,256,800,3),dtype=np.float32) # image place-holders
    Y=np.empty((self.batch_size,256,800,1),dtype=np.int8)# 1 mask place-holders
    for i,id in enumerate(batch):
      X[i,] = Image.open('/content/steel/train_images/' + str(self.df['image_id'].loc[id])).resize((800,256))
      Y[i,:,:,0]=rle_to_mask(self.df['rle'].loc[id])

    t=random.choice([0,0,20,30,40])
    z=random.choice([0.9,1])
    flip=random.choice(['True','False'])
    """
    https://theailearner.com/2019/07/06/imagedatagenerator-apply_transform-method/
    https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/image/ImageDataGenerator
    tx and ty: These are the shifts in the vertical and the horizontal directions respectively.
    For instance,tx=20 will shift the image vertically by 20 pixels
    zx and zy: This zooms the image in the vertical and horizontal directions respectively.
    If less than 1, the image is zoomed in otherwise zoomed out."""
    param={'flip_horizontal':flip,'flip_vertical':flip,'tx':t,'ty':t,'zx':z,'zy':z,}
    for i,e in enumerate(X):
      X[i] = train_datagen.apply_transform(e,transform_parameters=param)
    for i,f in enumerate(Y):
      Y[i] = train_datagen.apply_transform(f,transform_parameters=param)
    # preprocess input
    if self.preprocess!=None: X = self.preprocess(X)

    return X,Y

In [136]:
class test_DataGenerator(keras.utils.all_utils.Sequence):
  def __init__(self,dataframe,batch_size=1,shuffle=False,preprocess=None,info={}):
    self.batch_size = batch_size
    self.df = dataframe
    self.indices = self.df.index.tolist()
    self.preprocess = preprocess
    self.shuffle = shuffle
    self.on_epoch_end()

  def __len__(self):
    return len(self.indices) // (self.batch_size)

  def __getitem__(self, index):
    index = self.index[index * self.batch_size:(index + 1) * self.batch_size]
    batch = [self.indices[k] for k in index]
    X, y = self.__get_data(batch)
    return X, y

  def on_epoch_end(self):
    self.index = np.arange(len(self.indices))
    if self.shuffle == True:
      np.random.shuffle(self.index)

  def __get_data(self, batch):
    X = np.empty((self.batch_size,256,800,3),dtype=np.float32) # image place-holders
    Y = np.empty((self.batch_size,256,800,1),dtype=np.int8)# 1 mask place-holders
    for i, id in enumerate(batch):
      X[i,] = Image.open('/content/steel/train_images/' + str(self.df['image_id'].loc[id])).resize((800,256))
      Y[i,:,:,0]=rle_to_mask(self.df['rle'].loc[id])
    # preprocess input
    if self.preprocess!=None: X = self.preprocess(X)

    return X,Y

In [137]:
#https://segmentation-models.readthedocs.io/en/latest/tutorial.html
#LOAD UNET WITH PRETRAINING FROM IMAGENET
#https://segmentation-models.readthedocs.io/en/latest/tutorial.html
preprocess=get_preprocessing('efficientnetb5')
model=Unet('efficientnetb5',classes=1,activation='sigmoid',encoder_weights='imagenet')
model._name="Segmentation_Model"
model.summary()

Model: "Segmentation_Model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_6 (InputLayer)           [(None, None, None,  0           []                               
                                 3)]                                                              
                                                                                                  
 stem_conv (Conv2D)             (None, None, None,   1296        ['input_6[0][0]']                
                                48)                                                               
                                                                                                  
 stem_bn (BatchNormalization)   (None, None, None,   192         ['stem_conv[0][0]']              
                                48)                                              

In [ ]:
train_batch=train_DataGenerator(train_1,shuffle=True,preprocess=preprocess)
valid_batch=test_DataGenerator(val_1,preprocess=preprocess)
log_dir=os.path.join("logs",datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard=tf.keras.callbacks.TensorBoard(log_dir=log_dir,histogram_freq=1,write_graph=True, write_grads=True)
checkpoint_filepath='/content/drive/MyDrive/segmentation_defect_1.h5'
model_checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_filepath,monitor='val_dice_coef',mode='max',save_best_only=True)
#https://keras.io/api/metrics/
#https://keras.io/api/losses/probabilistic_losses/#categorical_crossentropy-function
model.compile(optimizer='adam',loss=bce_dice_loss,metrics=[dice_coef])
callback=[model_checkpoint_callback,tensorboard]

#https://datascience.stackexchange.com/questions/34444/what-is-the-difference-between-fit-and-fit-generator-in-keras
history=model.fit(train_batch,validation_data=valid_batch,epochs=25,verbose=1,callbacks=callback)

Epoch 1/25
161/161 [==============================] - 5530s 34s/step - loss: 0.7944 - dice_coef: 0.3171 - val_loss: 1.0903 - val_dice_coef: 0.2303
Epoch 2/25
 95/161 [================>.............] - ETA: 35:50 - loss: 0.4853 - dice_coef: 0.5555

In [ ]:
%tensorboard --logdir logs

In [ ]:
print('Training Dataset:\n')
print(model.evaluate(test_DataGenerator(train_1,preprocess=preprocess),verbose=1))
print("="*100)
print('\nValidation Dataset:\n')
print(model.evaluate(test_DataGenerator(val_1,preprocess=preprocess),verbose=1))
print("="*100)
print('\nTest Dataset:\n')
print(model.evaluate(test_DataGenerator(test_1,preprocess=preprocess),verbose=1))

For validation,test datasets loss increases and metrics decreases as compared with train data thus better to train model for
more epochs (e.g. epoch =100) which might help to reduce loss and increase dice coefficient on unseen data.

In [ ]:
def rle2mask(rle):
  # CONVERT RLE TO MASK
  if (pd.isnull(rle))|(rle=='')|(rle=='-1'):
    return np.zeros((256,1600) ,dtype=np.uint8)

  height= 256
  width = 1600
  mask= np.zeros( width*height ,dtype=np.uint8)
  array = np.asarray([int(x) for x in rle.split()])
  starts = array[0::2]-1
  lengths = array[1::2]
  for index, start in enumerate(starts):
    mask[int(start):int(start+lengths[index])] = 1

  return mask.reshape( (height,width), order='F' )

In [ ]:
model=load_model('/content/drive/MyDrive/segmentation_defect_1.h5',custom_objects={'bce_dice_loss':bce_dice_loss,'dice_coef':dice_coef})

In [ ]:
def plot_mask(rle_defect,k,pred):
  train_folder_path='/content/steel/train_images/'
  # Create figure and axes
  fig,ax=plt.subplots(4,3,figsize=(14,9))
  fig.suptitle('Defect_'+str(k)+'_Images',fontsize=20,fontweight='bold')
  for i in range(4):
    image_id=rle_defect[i][0]
    rle=rle_defect[i][1]
    im=Image.open(train_folder_path+str(image_id))
    ax[i,0].imshow(im)
    ax[i,0].set_title(image_id)
    mask=rle2mask(rle)
    ax[i,1].imshow(mask)
    ax[i,1].set_title("Actual Mask for "+str(image_id))
    c1=Image.fromarray(pred[i][:,:,0])
    ax[i,2].imshow(np.array(c1.resize((1600,256)))>0.5)
    ax[i,2].set_title("Predicted Mask for "+str(image_id))
  fig.set_facecolor("tan")
  plt.show()

## Train Dataset

In [ ]:
train_preds=model.predict_generator(test_DataGenerator(train_1[10:14],preprocess=preprocess),verbose=1)

In [ ]:
plot_mask(train_1[10:14].values,1,train_preds)

## Validation Dataset

In [ ]:
val_preds=model.predict_generator(test_DataGenerator(val_1[10:14],preprocess=preprocess),verbose=1)

In [ ]:
plot_mask(val_1[10:14].values,1,val_preds)

## Test Dataset

In [ ]:
test_preds=model.predict_generator(test_DataGenerator(test_1[10:14],preprocess=preprocess),verbose=1)

In [ ]:
plot_mask(test_1[10:14].values,1,test_preds)

## Defect_2

In [ ]:
train_batch=train_DataGenerator(train_2,shuffle=True,preprocess=preprocess)
valid_batch=test_DataGenerator(val_2,preprocess=preprocess)
log_dir=os.path.join("logs",datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard=tf.keras.callbacks.TensorBoard(log_dir=log_dir,histogram_freq=1,write_graph=True,write_grads=True)
checkpoint_filepath='/content/drive/MyDrive/segmentation_defect_2.h5'
model_checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_filepath,monitor='val_dice_coef',mode='max',save_best_only=True)
#https://keras.io/api/metrics/
#https://keras.io/api/losses/probabilistic_losses/#categorical_crossentropy-function
model.compile(optimizer='adam',loss=bce_dice_loss,metrics=[dice_coef])
callback=[model_checkpoint_callback,tensorboard]
#https://datascience.stackexchange.com/questions/34444/what-is-the-difference-between-fit-and-fit-generator-in-keras
history=model.fit_generator(train_batch,validation_data=valid_batch,epochs=25,verbose=1,callbacks=callback)

In [ ]:
%tensorboard --logdir logs

In [ ]:
model=load_model('/content/drive/MyDrive/segmentation_defect_2.h5',custom_objects={'bce_dice_loss':bce_dice_loss,'dice_coef':dice_coef})

In [ ]:
print('Training Dataset:\n')
print(model.evaluate(test_DataGenerator(train_2,preprocess=preprocess),verbose=1))
print("="*100)
print('\nValidation Dataset:\n')
print(model.evaluate(test_DataGenerator(val_2,preprocess=preprocess),verbose=1))
print("="*100)
print('\nTest Dataset:\n')
print(model.evaluate(test_DataGenerator(test_2,preprocess=preprocess),verbose=1))

One can observe the value of loss and metrics are similar for train,validation and test datasets thus the model is not over-
fitting. On unseen data (test) dice coefficient increases and loss decreases as compared to train, validation datasets thus,
model is performing really good.

## Training Dataset

In [ ]:
train_preds=model.predict_generator(test_DataGenerator(train_2[10:14],preprocess=preprocess),verbose=1)

In [ ]:
plot_mask(train_2[10:14].values,2,train_preds)

## Validation Dataset

In [ ]:
val_preds=model.predict_generator(test_DataGenerator(val_2[10:14],preprocess=preprocess),verbose=1)

In [ ]:
plot_mask(val_2[10:14].values,2,val_preds)

## Test Datset

In [ ]:
test_preds=model.predict_generator(test_DataGenerator(test_2[10:14],preprocess=preprocess),verbose=1)

In [ ]:
plot_mask(test_2[10:14].values,2,test_preds)

## Defect 3

In [ ]:
train_batch=train_DataGenerator(train_3,shuffle=True,preprocess=preprocess)
valid_batch=test_DataGenerator(val_3,preprocess=preprocess)
log_dir=os.path.join("logs",datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard=tf.keras.callbacks.TensorBoard(log_dir=log_dir,histogram_freq=1,write_graph=True,write_grads=True)
checkpoint_filepath='/content/drive/MyDrive/segmentation_defect_3.h5'
model_checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_filepath,monitor='val_dice_coef',mode='max',save_best_only=True)
#https://keras.io/api/metrics/
#https://keras.io/api/losses/probabilistic_losses/#categorical_crossentropy-function
model.compile(optimizer='adam',loss=bce_dice_loss,metrics=[dice_coef])
callback=[model_checkpoint_callback,tensorboard]
#https://datascience.stackexchange.com/questions/34444/what-is-the-difference-between-fit-and-fit-generator-in-keras
history=model.fit_generator(train_batch,validation_data=valid_batch,epochs=6,verbose=1,callbacks=callback)

In [ ]:
%tensorboard --logdir logs

In [ ]:
model=load_model('/content/drive/MyDrive/segmentation_defect_3.h5',custom_objects={'bce_dice_loss':bce_dice_loss,'dice_coef':dice_coef})

In [ ]:
print('Training Dataset:\n')
print(model.evaluate(test_DataGenerator(train_3,preprocess=preprocess),verbose=1))
print("="*100)
print('\nValidation Dataset:\n')
print(model.evaluate(test_DataGenerator(val_3,preprocess=preprocess),verbose=1))
print("="*100)
print('\nTest Dataset:\n')
print(model.evaluate(test_DataGenerator(test_3,preprocess=preprocess),verbose=1))

One can observe the value of loss and metrics are similar for train,validation and test datasets thus the model is not over-
fitting. On unseen data (test) dice coefficient slightly increases and loss decreases as compared to train, validation
datasets thus, model is performing really good.

## Training Dataset

In [ ]:
train_preds=model.predict_generator(test_DataGenerator(train_3[12:16],preprocess=preprocess),verbose=1)

In [ ]:
plot_mask(train_3[12:16].values,3,train_preds)

## Validation Dataset


In [ ]:
val_preds=model.predict_generator(test_DataGenerator(val_3[12:16],preprocess=preprocess),verbose=1)

In [ ]:
plot_mask(val_3[12:16].values,3,val_preds)

## Test Dataset

In [ ]:
test_preds=model.predict_generator(test_DataGenerator(test_3[12:16],preprocess=preprocess),verbose=1)

In [ ]:
plot_mask(test_3[12:16].values,3,test_preds)

## Defect 4

In [ ]:
train_batch=train_DataGenerator(train_4,shuffle=True,preprocess=preprocess)
valid_batch=test_DataGenerator(val_4,preprocess=preprocess)
log_dir=os.path.join("logs",datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard=tf.keras.callbacks.TensorBoard(log_dir=log_dir,histogram_freq=1,write_graph=True,write_grads=True)
checkpoint_filepath='/content/drive/MyDrive/segmentation_defect_4.h5'
model_checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_filepath,monitor='val_dice_coef',mode='max',save_best_only=True)
#https://keras.io/api/metrics/
#https://keras.io/api/losses/probabilistic_losses/#categorical_crossentropy-function
model.compile(optimizer='adam',loss=bce_dice_loss,metrics=[dice_coef])
callback=[model_checkpoint_callback,tensorboard]
#https://datascience.stackexchange.com/questions/34444/what-is-the-difference-between-fit-and-fit-generator-in-keras
history=model.fit_generator(train_batch,validation_data=valid_batch,epochs=25,verbose=1,callbacks=callback)

In [ ]:
%tensorboard --logdir logs

In [ ]:
model=load_model('/content/drive/MyDrive/segmentation_defect_4.h5',custom_objects={'bce_dice_loss':bce_dice_loss,'dice_coef':dice_coef})

In [ ]:
print('Training Dataset:\n')
print(model.evaluate(test_DataGenerator(train_4,preprocess=preprocess),verbose=1))
print("="*100)
print('\nValidation Dataset:\n')
print(model.evaluate(test_DataGenerator(val_4,preprocess=preprocess),verbose=1))
print("="*100)
print('\nTest Dataset:\n')
print(model.evaluate(test_DataGenerator(test_4,preprocess=preprocess),verbose=1))

For validation,test datasets loss slightly increases and metrics slightly decreases as compared with train data but overall
values are similar thus model is not over-fitting. For better results on unseen data train model for more epochs (e.g. epoch
=100) based on the available resources (memory).

## Training Dataset

In [ ]:
train_preds=model.predict_generator(test_DataGenerator(train_4[14:18],preprocess=preprocess),verbose=1)

In [ ]:
plot_mask(train_4[14:18].values,4,train_preds)

## Validation Dataset

In [ ]:
val_preds=model.predict_generator(test_DataGenerator(val_4[14:18],preprocess=preprocess),verbose=1)

In [ ]:
plot_mask(val_4[14:18].values,4,val_preds)

## Test Dataset

In [ ]:
test_preds=model.predict_generator(test_DataGenerator(test_4[14:18],preprocess=preprocess),verbose=1)

In [ ]:
plot_mask(test_4[14:18].values,4,test_preds)